# Step 1: 
Connect to DuckDB (in-memory by default, or specify a file path)

In [ ]:
import duckdb
# Initialize DuckDB connection
con = duckdb.connect()

# Define the base path
base_path = '/Users/mlima/Code/mirianlima.github.io/blog/posts/2024-08-12-duckdb-motherduck/data/'

# Step 2: 
Load CSV data into DuckDB using Relations

In [ ]:
c_age1 = con.read_csv(f'{base_path}c_age1.csv')
c_dem = con.read_csv(f'{base_path}c_dem.csv')

# Step 3: 
Filter, group, aggregate and pivot wide the `c_age1` data using Relations

In [ ]:
agg_age_data = (
    c_age1
    .filter("age_labor IN ('Younger Prime-age Workers (25-34)', 'Older Prime-age Workers (35-54)')")
    .project("""
        iso3_code, 
        time,
        CASE 
            WHEN age_labor = 'Younger Prime-age Workers (25-34)' THEN 'prime_younger'
            WHEN age_labor = 'Older Prime-age Workers (35-54)' THEN 'prime_older'
        END AS age_labor_group,
        pop_total
    """)
    .aggregate("""
        iso3_code, 
        time, 
        age_labor_group, 
        SUM(pop_total) AS total_population
    """)
    .aggregate("""
        iso3_code, 
        time,
        SUM(CASE WHEN age_labor_group = 'prime_younger' THEN total_population ELSE 0 END) AS prime_younger,
        SUM(CASE WHEN age_labor_group = 'prime_older' THEN total_population ELSE 0 END) AS prime_older
    """)
 )

# Step 4: 
Merging the pivoted data with `c_dem` using `iso3_code` and `time`

In [ ]:
final_data = (
    agg_age_data
    .set_alias("p")
    .join(c_dem.set_alias("d"), "p.iso3_code = d.iso3_code AND p.time = d.time")
)

# Step 5: 
Show a sample of the joined data

In [ ]:
final_data.limit(6).show()

# Step 6: 
Write the final data to CSV and parquet files

CSV

In [49]:

# Write the final data to a CSV file with custom options
final_data.write_csv(
    file_name=f'{base_path}c_dem_premium.csv',
    sep=',',           # Delimiter
    na_rep='NA',       # Representation of missing values
    header=True,       # Include header in the output
    encoding='utf-8',  # Encoding of the output file
    compression='gzip',  # Compression for the output file (optional)
    overwrite=True      # Overwrite the file if it exists
)

Parquet

In [51]:
# Write the final data to a Parquet file with custom options
final_data.write_parquet(
    file_name=f'{base_path}c_dem_premium.parquet',
    compression='snappy'  # Compression type (e.g., 'snappy', 'gzip', 'brotli')
)
